In [1]:
from typing_extensions import final
import numpy as np
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output
import math
import sys
import cv2
import os
from PIL import Image

In [ ]:
np.exp()

In [13]:
label_to_color ={0: (1,1,1),
 1: (1, 0, 0),
 2: (0.737, 0.561, 0.561),
 3: (0.255, 0.412, 0.882),
 4: (0.545, 0.271, 0.0745),
 5: (0.98, 0.502, 0.447),
 6: (0.98, 0.643, 0.376),
 7: (0.18, 0.545, 0.341),
 8: (0.502, 0, 0.502),
 9: (0.627, 0.322, 0.176),
 10:(0.753, 0.753, 0.753),
 11:(0.529, 0.808, 0.922),
 12:(0.416, 0.353, 0.804),
 13:(0.439, 0.502, 0.565),
 14:(0.784, 0.302, 0.565),
 15:(0.867, 0.627, 0.867),
 16:(0, 1, 0.498),
 17:(0.275, 0.51, 0.706),
 18:(0.824, 0.706, 0.549),
 19:(0, 0.502, 0.502),
 20:(0.847, 0.749, 0.847),
 21:(1, 0.388, 0.278),
 22:(0.251, 0.878, 0.816),
 23:(0.933, 0.51, 0.933),
 24:(0.961, 0.871, 0.702)}


def arrangement(a, b, object_name):
    if object_name=='cow' or object_name=='sheep':
        p = [10, 11, 13, 12, 14, 15, 8, 9, 0, 7, 3, 4, 5, 6, 1, 2]
    elif object_name=='bird':
        p = [13, 14, 15, 4, 12, 8, 9, 10, 11, 5, 6, 7, 0, 1, 2, 3]
    elif object_name=='person':
        p = [10, 11, 13, 12, 15, 14, 9, 0, 7, 8, 5, 6, 1, 2, 3, 4]
    elif object_name=='cat':
        p = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5]
    elif object_name=='dog':
        p = [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 0, 1, 2, 3, 4, 5]
    elif object_name=='horse':
        p = [10, 11, 13, 12, 14, 15, 8, 9, 0, 7, 3, 4, 5, 6, 1, 2]
    else:
      print("error")
    return a[p], b[p], p

def label_2_image(img):
  rgb_img = np.zeros((img.shape[0],img.shape[1], 3)) 
  for key in label_to_color.keys():
      rgb_img[img == key] = label_to_color[key]
  return rgb_img

In [129]:
def mask_to_canvas(masks, bbx, obj_class, img_idx, prefix):
    max_num_nodes=16
    canvas_size=550
    class_dict = {0:'cow',1:'sheep',2:'bird',3:'person',4:'cat',5:'dog',6:'horse'}
    obj = class_dict[np.argmax(obj_class)]
    canvas = np.zeros((canvas_size, canvas_size), dtype= 'float32')
    bb_in, mmx, mapping = arrangement(bbx, masks, class_dict[np.argmax(obj_class)])
    for i in range(max_num_nodes):
        index = 0
        if(bb_in[i] != np.array([0, 0, 0, 0, 0])).all():
            index = mapping[i]
            _, x_min, y_min, x_max, y_max = np.round(bb_in[i]*canvas_size)           
            if(x_min > x_max):
                x_max, x_min = x_min, x_max
            if(y_min > y_max):
                y_max, y_min = y_min, y_max
            if x_max-x_min >=1 and y_max-y_min>=1:
                x, y = int(x_max-x_min), int(y_max-y_min)
                gen_mask = np.zeros((64, 64))
                gen_mask[np.where(np.squeeze(mmx[i])>=0.45)] = (i+1)
                canvas[int(y_min):int(y_max), int(x_min):int(x_max)] = np.maximum(canvas[int(y_min):int(y_max), int(x_min):int(x_max)],
                                                                                  cv2.resize(gen_mask, (x, y)))

    sza = 10
    plt.figure(num=None, figsize=(sza, sza))
    plt.axis('off')
    plt.imshow(label_2_image(canvas))
    plt.savefig('D:/meronym_data/runs/mask_generation_model_tf2_reconstruction/lr00001/maskResults/'+obj+'/'+str(img_idx)+ '_'+prefix+'.png')
    plt.close()

In [55]:
class_dict = {0:'cow',1:'sheep',2:'bird',3:'person',4:'cat',5:'dog',6:'horse'}
for key, value in class_dict.items():
    path = 'D:/meronym_data/runs/mask_generation_model_tf2_reconstruction/lr00001/maskResults/'+value+'/'
    
    if not os.path.isdir(path):
        os.mkdir(path)

In [21]:
file_postfix = '_combined_mask_data'

outfile = 'D:/meronym_data/genearted_masks.npy'
with open(outfile, 'rb') as pickle_file:
    masks_gen_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/X_val'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    X_train_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/class_v_val'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    class_v_val = pickle.load(pickle_file)

outfile = 'D:/meronym_data/masks_val'+file_postfix+'.np'
with open(outfile, 'rb') as pickle_file:
    masks_val = pickle.load(pickle_file)

In [130]:
prefix = 'gen'
for idx, (msk, bbx, obj_cls) in enumerate(zip(masks_gen_val[:10], X_train_val[:10], class_v_val[:10])):
    mask_to_canvas(msk, bbx, obj_cls, idx, prefix)

None
None
None
None
None
None
None
None
None
None
